# Differential drive robot (i.e., "segbot") in artificial gravity

Import modules and configure the notebook.

In [4]:
# This module is part of the python standard library
import time

# These modules are part of other existing libraries
import numpy as np
import sympy as sym
from scipy import linalg
from scipy import signal
import matplotlib.pyplot as plt

# This is my own script (it is an interface to the pybullet simulator)
import ae353_segbot

# I often go back and forth between making changes to my scripts and to
# the notebook in which they are used. One "gotcha" is that notebooks only
# import modules or scripts ONCE. Subsequent imports don't do anything, and
# in particular won't reflect any changes I've made to my scripts. To make
# sure that I'm working with the "latest version of my code, I use this bit
# of magic, which forces the notebook to "reload" my script:
import importlib
importlib.reload(ae353_segbot)

<module 'ae353_segbot' from '/Users/patrickthornton/Desktop/DP1 - ae353/ae353-sp22/projects/02_segbot/ae353_segbot.py'>

Create an instance of the `Simulator` class, which is an interface to the [pybullet](http://pybullet.org) simulation engine.

There are several optional arguments:

* `display` (`True` or `False`) is whether or not to show the simulation window;
* `width` and `height` (positive integers, default `640` and `480`, respectively) is the weight and height of the simulation window (if shown) in pixels;
* `bumpy` (`True` or `False`) is whether or not the station is "bumpy".

**You must evaluate this cell only *once*.** If you want to start fresh with a new simulator, you must do `Kernel -> Restart` from the notebook menu first. If you evaluate this cell more than once without a call to `Kernel -> Restart` then you may get strange behavior that is hard to debug.

In [ ]:
simulator = ae353_segbot.Simulator(
    display=True,
    width=480,
    height=320,
    bumpy=True,
)

### Define parameter values consistent with the URDF file:

In [13]:
### Define physical constants, consistent with the URDF file:
# Dimensions of chassis
dx = 0.4
dy = 0.6
dz = 0.8

# Distance between axle and COM of chassis
h = 0.3

# Half-distance between wheels
a = 0.7 / 2

# Mass of chassis
mb = 12.

# MOI of chassis
Jbx = (mb / 12) * (dy**2 + dz**2)
Jby = (mb / 12) * (dx**2 + dz**2)
Jbz = (mb / 12) * (dx**2 + dy**2)

# Radius of each wheel
r = 0.325

# Width of each wheel
hw = 0.075

# Mass of each wheel
mw = 1.2

# MOI of each wheel
Jw = (mw / 2) * r**2
Jwt = (mw / 12) * (3 * r**2 + hw**2)

# Total mass
m = mb + 2 * mw

# Total MOI
Jx = Jbx + 2 * Jwt
Jy = Jby
Jz = Jbz + 2 * Jwt

# Station parameters
station_velocity = -0.5 # <-- FIXME (change the velocity to change gravity)
station_radius = 20.    # <-- radius in meters of inside surface of station

# Acceleration of artifical gravity
g = station_velocity**2 * station_radius

### Convert all physical constants to rational numbers:
# Dimensions
h = sym.nsimplify(h)
a = sym.nsimplify(a)
r = sym.nsimplify(r)

# Masses
mb = sym.nsimplify(mb)
mw = sym.nsimplify(mw)
m = sym.nsimplify(m)

# MOIs
Jx = sym.nsimplify(Jx)
Jy = sym.nsimplify(Jy)
Jz = sym.nsimplify(Jz)
Jw = sym.nsimplify(Jw)
Jwt = sym.nsimplify(Jwt)

# Gravity
g = sym.nsimplify(g)

### Define variables
(e_l,
 e_h,
 v,
 w,
 theta,
 thetadot,
 phi,
 phidot,
 tau_R,
 tau_L) = sym.symbols('e_l, e_h, v, w, theta, thetadot, phi, phidot, tau_R, tau_L', real=True)

### Compute equations of motion, excluding lateral and heading errors (see Tuttle, 2014):
M = sym.Matrix([[m + 2 * Jw / r**2, 0, mb * h * sym.cos(theta)],
                [0, (Jx + mb * h**2) * sym.sin(theta)**2 + Jz * sym.cos(theta)**2 + (2 * Jw * a**2 / r**2) + 2 * mw * a**2, 0],
                [mb * h * sym.cos(theta), 0, Jy * mb * h**2]])
N = sym.Matrix([[mb * h * (w**2 + thetadot**2) * sym.sin(theta)],
                [-2 * (Jx - Jz + m * h**2) * sym.cos(theta) * sym.sin(theta) * w * thetadot - mb * h * sym.sin(theta) * v * w],
                [(Jx - Jz + mb * h**2) * sym.cos(theta) * sym.sin(theta) * w**2 + mb * g * h * sym.sin(theta)]])
R = sym.Matrix([[1 / r, 1 / r],
                [-a / r, a / r],
                [-1, -1]])
f = sym.simplify(M.inv() * (N + R * sym.Matrix([[tau_L], [tau_R]])))

### Compute full equations of motion (define right side of ODEs):
f = sym.Matrix([[v * sym.sin(e_h)],
                [w],
                [f]])
### note: should this be phi or theta?? ###
f = f.row_insert(4, sym.Matrix([[phi]]))
f

Matrix([
[                                                                                                                                                                           v*sin(e_h)],
[                                                                                                                                                                                    w],
[        -(1200*tau_L + 1200*tau_R + 1404*(thetadot**2 + w**2)*sin(theta) + 65*(50*tau_L + 50*tau_R - 39*w**2*sin(2*theta) - 900*sin(theta))*cos(theta)/2)/(5850*cos(theta)**2 - 6084)],
[                                                                    32*(-875*tau_L + 875*tau_R - 1443*thetadot*w*sin(2*theta) - 2925*v*w*sin(theta))/(13*(3120*sin(theta)**2 + 2051))],
[                                                                                                                                                                                  phi],
[5*(4225*tau_L + 4225*tau_R - 6591*w**2*sin(2*theta)/2 + 30*(100*t

In [14]:
f_num = sym.lambdify([e_l, e_h, v, w, theta, phi, phidot], f)
f_num(0, 0, 0, 0, 0, 0, 0)

array([[0.0],
       [0],
       [12.0726495726496*tau_L + 12.0726495726496*tau_R],
       [-1.05014439485429*tau_L + 1.05014439485429*tau_R],
       [0],
       [-51.460113960114*tau_L - 51.460113960114*tau_R]], dtype=object)

### Define a function to return a state-space model for a given choice of equilibrium point.

In [ ]:
def get_model(e_l_e,
              e_h_e,
              v_e,
              w_e,
              theta_e,
              phi_e,
              phidot_e,
              tau_R_e,
              tau_L_e):
    ### Define physical constants, consistent with the URDF file:
    # Dimensions of chassis
    dx = 0.4
    dy = 0.6
    dz = 0.8

    # Distance between axle and COM of chassis
    h = 0.3

    # Half-distance between wheels
    a = 0.7 / 2

    # Mass of chassis
    mb = 12.

    # MOI of chassis
    Jbx = (mb / 12) * (dy**2 + dz**2)
    Jby = (mb / 12) * (dx**2 + dz**2)
    Jbz = (mb / 12) * (dx**2 + dy**2)

    # Radius of each wheel
    r = 0.325

    # Width of each wheel
    hw = 0.075

    # Mass of each wheel
    mw = 1.2

    # MOI of each wheel
    Jw = (mw / 2) * r**2
    Jwt = (mw / 12) * (3 * r**2 + hw**2)

    # Total mass
    m = mb + 2 * mw

    # Total MOI
    Jx = Jbx + 2 * Jwt
    Jy = Jby
    Jz = Jbz + 2 * Jwt

    # Station parameters
    station_velocity = -0.5 # <-- FIXME (change the velocity to change gravity)
    station_radius = 20.    # <-- radius in meters of inside surface of station

    # Acceleration of artifical gravity
    g = station_velocity**2 * station_radius

    ### Convert all physical constants to rational numbers:
    # Dimensions
    h = sym.nsimplify(h)
    a = sym.nsimplify(a)
    r = sym.nsimplify(r)

    # Masses
    mb = sym.nsimplify(mb)
    mw = sym.nsimplify(mw)
    m = sym.nsimplify(m)

    # MOIs
    Jx = sym.nsimplify(Jx)
    Jy = sym.nsimplify(Jy)
    Jz = sym.nsimplify(Jz)
    Jw = sym.nsimplify(Jw)
    Jwt = sym.nsimplify(Jwt)

    # Gravity
    g = sym.nsimplify(g)

    ### Define variables
    (e_l,
     e_h,
     v,
     w,
     theta,
     phi,
     phidot,
     tau_R,
     tau_L) = sym.symbols('e_l, e_h, v, w, theta, phi, phidot, tau_R, tau_L', real=True)

    ### Compute equations of motion, excluding lateral and heading errors (see Tuttle, 2014):
    M = sym.Matrix([[m + 2 * Jw / r**2, 0, mb * h * sym.cos(theta)],
                    [0, (Jx + mb * h**2) * sym.sin(theta)**2 + Jz * sym.cos(theta)**2 + (2 * Jw * a**2 / r**2) + 2 * mw * a**2, 0],
                    [mb * h * sym.cos(theta), 0, Jy * mb * h**2]])
    N = sym.Matrix([[mb * h * (w**2 + phi**2) * sym.sin(theta)],
                    [-2 * (Jx - Jz + m * h**2) * sym.cos(theta) * sym.sin(theta) * w * phi - mb * h * sym.sin(theta) * v * w],
                    [(Jx - Jz + mb * h**2) * sym.cos(theta) * sym.sin(theta) * w**2 + mb * g * h * sym.sin(theta)]])
    R = sym.Matrix([[1 / r, 1 / r],
                    [-a / r, a / r],
                    [-1, -1]])
    f = sym.simplify(M.inv() * (N + R * sym.Matrix([[tau_L], [tau_R]])))

    ### Compute full equations of motion (define right-hand side of ODEs):
    f = sym.Matrix([[v * sym.sin(e_h)],
                    [w],
                    [f]])
    f = f.row_insert(4, sym.Matrix([[phi]]))
    
    # Convert floating-point to rational numbers
    f = sym.nsimplify(f, rational=True)
    
### continue ###

    # Verify equilibrium point
    f_num = sym.lambdify([e_l, e_h, v, w, theta, phi, phidot, tau_R, tau_L], f)
    if not np.allclose(f_num(e_l_e, e_h_e, v_e, w_e, theta_e, phi_e, phidot_e, tau_R_e, tau_L_e), 0.):
        raise Exception('equilibrium point is invalid')
    
    # Find A and B in symbolic form
    A_sym = f.jacobian([q1, v1, q2, v2])
    B_sym = f.jacobian([tau])
    
    # Create lambda functions to allow numerical evaluation of A and B
    A_num = sym.lambdify([q1, v1, q2, v2, tau], A_sym)
    B_num = sym.lambdify([q1, v1, q2, v2, tau], B_sym)
    
    # Find A and B in numeric form (making sure the result is floating-point)
    A = A_num(q1e, q2e, v1e, v2e, taue).astype(float)
    B = B_num(q1e, q2e, v1e, v2e, taue).astype(float)
    
    # Return A and B
    return A, B

Define a controller that maps sensor measurements to actuator commands. By default, it chooses to apply zero torque to the gimbal. Modify the `run` function to make the controller behave differently.

In [ ]:
class Controller:
    def __init__(self):
        pass
    
    def reset(self):
        pass
    
    def run(
            self,
            t,
            lateral_error,
            heading_error,
            forward_speed,
            turning_rate,
            pitch_angle,
            pitch_rate,
        ):
        
        # FIXME: replace the following lines
        right_wheel_torque = 0.
        left_wheel_torque = 0.
        
        return right_wheel_torque, left_wheel_torque

Create an instance of the controller.

In [ ]:
controller = Controller()

Reset the simulator with given initial conditions. You may also change the angular velocity of the station if you like, but - beware! - doing so will change the amount of artificial gravity.

In [ ]:
simulator.reset(
    initial_speed=1.,
    initial_lateral_error=0.,
    initial_heading_error=0.,
    initial_pitch=0.,
    station_velocity=-0.5,
)

Reset the controller.

In [ ]:
controller.reset()

Choose a camera view from the following options:
* `camera_sideview()`
* `camera_backview()`
* `camera_wideview()`

In [ ]:
simulator.camera_backview()

Run the simulator.

In [ ]:
data = simulator.run(
    controller,           # <-- required (an instance of your Controller class)
    max_time=5.0,         # <-- optional (how long you want to run the simulation in seconds)
    data_filename=None,   # <-- optional (save data to this file, e.g., 'my_data.json')
    video_filename=None,  # <-- optional (save video to this file, e.g., 'my_video.mov')
)

Get, show, and save a snapshot of the simulation after the last time step.

In [ ]:
# Get snapshot as height x width x 4 numpy array of RGBA values
rgba = simulator.snapshot()

# Display snapshot
plt.figure(figsize=(8, 8))
plt.imshow(rgba)

# Save snapshot
plt.imsave('my_snapshot.png', rgba)

Plot the results.

In [ ]:
# Create a figure with three subplots, all of which share the same x-axis
fig, (ax_pos, ax_vel, ax_pitch, ax_torque) = plt.subplots(4, 1, figsize=(8, 8), sharex=True)

# Plot lateral and heading error
ax_pos.plot(
    data['t'], data['lateral_error'],
    label='lateral_error (m)', linewidth=4,
)
ax_pos.plot(
    data['t'], data['heading_error'],
    '--', label='heading_error (rad)', linewidth=4,
)
ax_pos.grid()
ax_pos.legend(fontsize=16)
ax_pos.tick_params(labelsize=14)

# Plot forward speed and turning rate
ax_vel.plot(
    data['t'], data['forward_speed'],
    label='forward speed (m/s)', linewidth=4,
)
ax_vel.plot(
    data['t'], data['turning_rate'],
    '--', label='turning rate (rad / s)', linewidth=4,
)
ax_vel.grid()
ax_vel.legend(fontsize=16)
ax_vel.tick_params(labelsize=14)

# Plot pitch angle and pitch rate
ax_pitch.plot(
    data['t'], data['pitch_angle'],
    label='pitch angle (rad)', linewidth=4,
)
ax_pitch.plot(
    data['t'], data['pitch_rate'],
    '--', label='pitch rate (rad)', linewidth=4,
)
ax_pitch.grid()
ax_pitch.legend(fontsize=16)
ax_pitch.tick_params(labelsize=14)

# Plot torques
ax_torque.plot(
    data['t'], data['right_wheel_torque_command'],
    label='right wheel torque command (N-m)', linewidth=4,
)
ax_torque.plot(
    data['t'], data['right_wheel_torque'],
    '--', label='right wheel torque (N-m)', linewidth=4,
)
ax_torque.plot(
    data['t'], data['left_wheel_torque_command'],
    label='left wheel torque command (N-m)', linewidth=4,
)
ax_torque.plot(
    data['t'], data['left_wheel_torque'],
    '--', label='left wheel torque (N-m)', linewidth=4,
)
ax_torque.plot(
    data['t'], -np.ones_like(data['t']) * simulator.tau_max,
    ':', linewidth=4, color='C4', zorder=0,
)
ax_torque.plot(
    data['t'], np.ones_like(data['t']) * simulator.tau_max,
    ':', linewidth=4, color='C4', zorder=0,
)
ax_torque.grid()
ax_torque.legend(fontsize=16)
ax_torque.tick_params(labelsize=14)
ax_torque.set_ylim(-1.2 * simulator.tau_max, 1.2 * simulator.tau_max)

# Set x-axis properties (only need to do this on the last
# subplot since all subplots share the same x-axis)
ax_torque.set_xlabel('time (s)', fontsize=20)
ax_torque.set_xlim([data['t'][0], data['t'][-1]])

# Make the arrangement of subplots look nice
fig.tight_layout()

Save the figure (without transparent background).

In [ ]:
fig.savefig('my_figure.png', facecolor='white', transparent=False)